In [1]:
import pandas as pd
import numpy as np
pd.set_option('mode.chained_assignment', None)
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import roc_curve, auc
from sklearn import svm 

In [2]:
combined = pd.read_csv("../Personal Response_to_Resistance/Input Data/clean_data.csv", header=0)
#combined = pd.read_csv('clean_data.csv')
combined.head()

/Users/brinaseidel/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,date_time,date_x,time,loc_t ype,zip_code,council_district,apd_sector,apd_district,latitude,longtitude,...,P0030006_pct,P0030007_pct,P0030008_pct,P0040003_pct,P0200002_pct,P0250002_pct,P0190007_pct,B19113_001E,B17001_001E_pct,C18120_003E_pct
0,2014-10-01 20:19:00,10/01/2014,2019.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.315506,-97.690678,...,0.0004,0.211769,0.064652,0.529424,0.402545,0.130133,0.337189,35600.0,0.994701,0.88417
1,2014-10-05 16:33:00,10/05/2014,1633.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.315506,-97.690678,...,0.0004,0.211769,0.064652,0.529424,0.402545,0.130133,0.337189,35600.0,0.994701,0.88417
2,2012-01-20 22:30:00,01/20/2012,2230.0,PARKING LOTS / GARAGE,78723.0,1.0,ID,1,30.320667,-97.687671,...,0.0004,0.211769,0.064652,0.529424,0.402545,0.130133,0.337189,35600.0,0.994701,0.88417
3,2011-01-01 20:00:00,01/01/2011,2000.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.320667,-97.687671,...,0.0004,0.211769,0.064652,0.529424,0.402545,0.130133,0.337189,35600.0,0.994701,0.88417
4,2012-03-06 05:00:00,03/06/2012,500.0,RESIDENCE / HOME,78723.0,1.0,ID,1,30.320667,-97.687671,...,0.0004,0.211769,0.064652,0.529424,0.402545,0.130133,0.337189,35600.0,0.994701,0.88417


In [3]:
# APD sector
apd_sector_dummies = pd.get_dummies(combined.apd_sector)
cols = []
for col in apd_sector_dummies.columns:
    cols.append("apd_sector"+str(col))
apd_sector_dummies.columns = cols

# Zip codes
zip_dummies = pd.get_dummies(combined.zip_code)
cols = []
for col in zip_dummies.columns:
    cols.append("zip"+str(int(col)))
zip_dummies.columns = cols

combined = pd.concat([combined, apd_sector_dummies, zip_dummies], axis=1)

# Location type
loc_type_dummies = pd.get_dummies(combined['loc_t ype'])
cols = []
for col in loc_type_dummies.columns:
    cols.append("loc_type"+str(col))
loc_type_dummies.columns = cols

Split into training and testing data

NOTE: Try taking out loc_type?

In [4]:
cols_to_use = ["temp_max", "temp_min", "prec_avg", 'new_year', 'mlk_day', 'wash_bday', 'mem_day', 'ind_day', 
      'labor_day', 'col_day', 'vet_day', 'thanksgiving', 'christmas', 'P0030001','P0030002_pct', 'P0030003_pct', 'P0030004_pct', 'P0030005_pct', 'P0030006_pct', 
      'P0040003_pct', 'P0130001', 'B19113_001E', 'B17001_001E_pct', 'C18120_003E_pct']
months = [col for col in combined if col.startswith('month') and col != "month"]
days = [col for col in combined if col.startswith('day') and col != "day_of_week"]
hours = [col for col in combined if col.startswith('hour') and col != "hour_of_day"]
zips = [col for col in combined if col.startswith('zip') and col != "zip_code"]
apd_sectors =  [col for col in combined if col.startswith('apd_sector') and col != "apd_sector"]
loc_type = [col for col in combined if col.startswith('loc_type')]
cols_to_use = cols_to_use + months + days + hours + zips + apd_sectors + loc_type
cols_to_use_y = cols_to_use + ["Target1"]

In [6]:
combined_nonmiss = combined[cols_to_use_y].dropna()
print('Records dropped because of nans: '+str(len(combined)-len(combined_nonmiss)))

Records dropped because of nans: 15634


In [7]:
X = combined_nonmiss[cols_to_use]
y = combined_nonmiss["Target1"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [8]:
# Check that the stratification worked
print("Training:")
print(y_train.value_counts(normalize=True))
print("\nTesting:")
print(y_test.value_counts(normalize=True))

Training:
0.0    0.987127
1.0    0.012873
Name: Target1, dtype: float64

Testing:
0.0    0.98713
1.0    0.01287
Name: Target1, dtype: float64


## Run models on training data

In [9]:
# Set up stratified k fold split
skf = StratifiedKFold(n_splits=5, random_state=400)
skf.get_n_splits(X_train, y_train)

5

In [10]:
# Logistic regression 
auc_lr = []
for train_index, test_index in skf.split(X_train, y_train):
    
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model = LogisticRegression(C=1e30).fit(X_train_fold.drop(['month1', 'hour0', 'day1', 'apd_sector1', 'zip76574'], axis=1), y_train_fold)
    
    y_pred_fold = model.predict_proba(X_test_fold.drop(['month1', 'hour0', 'day1', 'apd_sector1', 'zip76574'], axis=1))[:,1]
    fpr, tpr, thresholds = roc_curve(y_test_fold, y_pred_fold)
    auc_lr.append(auc(fpr, tpr))

auc_lr

[0.6056093879095984,
 0.6270041387567771,
 0.6215275234117839,
 0.612473816393459,
 0.6217097231565192]

In [11]:
# Naive bayes
auc_nb = []
for train_index, test_index in skf.split(X_train, y_train):
    
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model = BernoulliNB().fit(X_train_fold, y_train_fold)
    
    y_pred_fold = model.predict_proba(X_test_fold)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test_fold, y_pred_fold)
    auc_nb.append(auc(fpr, tpr))

auc_nb

[0.6894346829656525,
 0.6920072692797773,
 0.6976217270829276,
 0.6879365253121915,
 0.6937605327510444]

In [12]:
# Decision tree
auc_dt = []
for train_index, test_index in skf.split(X_train, y_train):
    
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    model = DecisionTreeClassifier(criterion="entropy").fit(X_train_fold, y_train_fold)
    
    y_pred_fold = model.predict_proba(X_test_fold)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test_fold, y_pred_fold)
    auc_dt.append(auc(fpr, tpr))

auc_dt

[0.5517940807912758,
 0.5556195368472145,
 0.5560494603633377,
 0.5545349292565165,
 0.5535502087327919]

In [ ]:
# SVM - THIS DOESN"T WORK (in a reasonable amount of time)!
auc_svm = []
for train_index, test_index in skf.split(X_train, y_train):
        
    X_train_fold, X_test_fold = X_train.iloc[train_index], X_train.iloc[test_index]
    y_train_fold, y_test_fold = y_train.iloc[train_index], y_train.iloc[test_index]
    
    svm_model = svm.SVC(kernel='linear')
    svm_model.fit(X_train_fold, y_train_fold)
    
    y_pred_fold = svm.predict_proba(X_test_fold)[:,1]
    fpr, tpr, thresholds = roc_curve(y_test_fold, y_pred_fold)
    auc_svm.append(auc(fpr, tpr))
    

Print the vars we have (just for reference)

In [19]:
for col in combined.columns:
    print("'"+col+"', ")


'date_time', 
'date_x', 
'time', 
'loc_t ype', 
'zip_code', 
'council_district', 
'apd_sector', 
'apd_district', 
'latitude', 
'longtitude', 
'location', 
'year', 
'Target1', 
'Target2', 
'temp_max', 
'temp_avg', 
'temp_min', 
'dew_max', 
'dew_avg', 
'dew_min', 
'hum_max', 
'hum_min', 
'wind_max', 
'wind_min', 
'pres_max', 
'pres_min', 
'prec_avg', 
'new_year', 
'mlk_day', 
'wash_bday', 
'mem_day', 
'ind_day', 
'labor_day', 
'col_day', 
'vet_day', 
'thanksgiving', 
'christmas', 
'month', 
'day_of_week', 
'hour_of_day', 
'month1', 
'month2', 
'month3', 
'month4', 
'month5', 
'month6', 
'month7', 
'month8', 
'month9', 
'month10', 
'month11', 
'month12', 
'day0', 
'day1', 
'day2', 
'day3', 
'day4', 
'day5', 
'day6', 
'hour0', 
'hour1', 
'hour2', 
'hour3', 
'hour4', 
'hour5', 
'hour6', 
'hour7', 
'hour8', 
'hour9', 
'hour10', 
'hour11', 
'hour12', 
'hour13', 
'hour14', 
'hour15', 
'hour16', 
'hour17', 
'hour18', 
'hour19', 
'hour20', 
'hour21', 
'hour22', 
'hour23', 
'tract', 
'P0030001', 